# Knowledge graph builder
In this notebook, we will build the knowledge graph from 3 datasources:
- **Extracted data**: These are the LLM extracted data, which contains the coulombic efficiency and the electrolyte information.
- **csv dataset**: This is the supplementary material from the paper Kim, Sang Cheol, et al. "Data-driven electrolyte design for lithium metal anodes." Proceedings of the National Academy of Sciences 120.10 (2023): e2214357120. https://doi.org/10.1073/pnas.2214357120.
- **papers**: These are the paper references in csv database, renamed by their reference id.

To protect the intellectual property of the authors, we will **NOT** release the csv dataset and papers. The built knowledge graph can be accessed through a guest account with:
`URI`: neo4j+s://4013e16e.databases.neo4j.io
`username`: guest
`password`: guest12345

In [1]:
import pickle
import os
import pandas as pd
from math import log10
from langchain.graphs import Neo4jGraph
from neo4j import GraphDatabase
from langchain_text_splitters import TokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os

KG_URI = "your_neo4j_uri"
KG_USER = "your_neo4j_user"
KG_PASSWORD = "your_neo4j_password"
data_path = "database/extracted_info/"
papers_path = "database/papers/"
csv_path = "database/csv/"

In [ ]:
extracted_data ={}
files = os.listdir(data_path)
for file in files:
    if file.endswith(".pkl"):
        with open(f"{data_path}/{file}", "rb") as f:
            extracted_data[file] = pickle.load(f)
extracted_data_list = []
for k,v in extracted_data.items():
    # print(k.split("_")[0])
    d = v['data']
    # if d is list
    if isinstance(d, list):
        for i in d:
            i['source'] = k.split("_")[0]
            extracted_data_list.append(i)
    # if d is dict
    else:
        d['source'] = k.split("_")[0]
    extracted_data_list.extend(d)

In [35]:
xlsx_name = "pnas.2214357120.sd01.xlsx"
dataset_paper = pd.read_excel(xlsx_name, sheet_name="Dataset")
# skip 1st col
dataset_paper= dataset_paper.iloc[1:,2:26]
dataset_paper.columns=["sol1_vol", "sol2_vol", "sol3_vol", "sol1_mol", "sol2_mol", "sol3_mol", "slt1_mol", "slt2_mol", "slt3_mol" ,"FC", "OC",	"FO",	"InOr",	"F",	"sF",	"aF",	"O",	"sO","aO",	"C",	"sC",	"aC",	"CE",	"LCE"]
dataset_paper.head()

,sol1_vol,sol2_vol,sol3_vol,sol1_mol,sol2_mol,sol3_mol,slt1_mol,slt2_mol,slt3_mol,FC,...,sF,aF,O,sO,aO,C,sC,aC,CE,LCE
1,1,0.05,NaN,11.754565,0.6855,NaN,1,NaN,NaN,0.013968,...,0.004138,0,0.249421,0.225276,0.024145,0.29623,0.29623,0,80,0.69897
2,0.5,0.5,0.02,7.49472,5.939297,0.313786,1,NaN,NaN,0.145478,...,0,0.038283,0.263151,0.263151,0,0.263151,0.263151,0,80,0.69897
3,0.5,0.5,0.02,7.49472,5.939297,0.313786,1,NaN,NaN,0.142634,...,0.012022,0.025541,0.259341,0.259341,0,0.263348,0.263348,0,80,0.69897
4,1,NaN,NaN,11.754565,NaN,NaN,1,NaN,NaN,0.239912,...,0,0.070254,0.212305,0.206451,0.005854,0.292831,0.275267,0.017563,80.6,0.712198
5,0.246207,0.753793,0.02,3.690498,7.284218,0.313786,1,NaN,NaN,0.145809,...,0,0.038295,0.216146,0.216146,0,0.262638,0.262638,0,82,0.744727


In [36]:
molecular_db = pd.read_excel(xlsx_name, sheet_name="Molecular Database", header=2)
solvent_db = molecular_db.iloc[:53,1:]
salt_db = pd.read_excel(xlsx_name, sheet_name="Molecular Database", header=57)
salt_db = salt_db.iloc[:,1:]

# replace nan with 0
solvent_db = solvent_db.fillna(0)
salt_db = salt_db.fillna(0)

solvent_db.head()

C:\Users\RiceCake\AppData\Local\Temp\ipykernel_2388\1282106779.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  solvent_db = solvent_db.fillna(0)


,Solvent,C,H,O,F,N,S,P,Unnamed: 9,B/Cl/As/Si etc.,MW,density (g/mL),mol/L,SMILES
0,DME (C4H10O2),4,10,2,0,0,0,0,0,0,90.120,0.868,9.631602,COCCOC
1,EC (C3H4O3),3,4,3,0,0,0,0,0,0,88.062,1.320,14.989439,C1COC(=O)O1
2,PC (C4H6O3),4,6,3,0,0,0,0,0,0,102.088,1.200,11.754565,CC1COC(=O)O1
3,DEC (C5H10O3),5,10,3,0,0,0,0,0,0,118.130,0.975,8.253619,O=C(OCC)OCC
4,DMC (C3H6O3),3,6,3,0,0,0,0,0,0,90.078,1.070,11.878594,COC(=O)OC


In [37]:
salt_db.head()

,Salt,C,H,O,F,N,S,P,Li,B/Cl/As/Si etc.,Unnamed: 11,SMILES,Unnamed: 13,Unnamed: 14
0,LiFSI,0.0,0.0,4.0,2.0,1.0,2.0,0.0,1.0,0.0,0.0,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.0,0.0
1,LiPF6,0.0,0.0,0.0,6.0,0.0,0.0,1.0,1.0,0.0,0.0,[Li+].F[P-](F)(F)(F)(F)F,0.0,0.0
2,LiTFSI,2.0,0.0,4.0,6.0,1.0,2.0,0.0,1.0,0.0,0.0,[Li+].C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,0.0,0.0
3,LiBOB,4.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,[Li+].[B-]12(OC(=O)C(=O)O1)OC(=O)C(=O)O2,0.0,0.0
4,LiDFP,0.0,0.0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,[Li+].[O-]P(=O)(F)F,0.0,0.0


In [38]:
paper_db = pd.read_excel(xlsx_name, sheet_name="References")
paper_db = paper_db.iloc[:,1:]
paper_db.columns = ['References', 'Title']
paper_db.head()

,References,Title
0,1,"Zheng, H. et al. Lithium Difluorophosphate-Bas..."
1,2,"Mogi, R. et al. Effects of Some Organic Additi..."
2,3,"Yu, Z. et al. Molecular design for electrolyte..."
3,4,"Dong, N. et al. A LiPO2F2/LiFSI dual-salt elec..."
4,5,"Kim, S. C. et al. Potentiometric Measurement t..."


In [39]:
solvent_name_id_map = {}
for idx, st in enumerate(solvent_db['Solvent'].to_list()):
    solvent_name_id_map[st.split(' ')[0]] = idx
# solvent_name_id_map

salt_name_id_map = {}
for idx, st in enumerate(salt_db['Salt'].to_list()):
    salt_name_id_map[st.split(' ')[0]] = idx
# salt_name_id_map

In [40]:
df = pd.DataFrame(["sol1_vol", "sol2_vol", "sol3_vol", "sol1_mol", "sol2_mol", "sol3_mol", "slt1_mol", "slt2_mol", "slt3_mol" ,"FC", "OC",	"FO",	"InOr",	"F",	"sF",	"aF",	"O",	"sO","aO",	"C",	"sC",	"aC",	"CE",	"LCE"])
data_new_list = []
for data in extracted_data_list:
    data_new = {}
    data_new['CE'] = data['ce']
    for idx, solvent in enumerate(data['electrolyte']['solvent_list']):
        data_new[f"sol{idx+1}_vol"] = solvent['volume_percent']
        data_new[f"sol{idx+1}_type"] = solvent['solvent']
    for idx, salt in enumerate(data['electrolyte']['salt_list']):
        data_new[f"slt{idx+1}_mol"] = salt['mol_L']
        data_new[f"slt{idx+1}_type"] = salt['salt']
    data_new['source'] = data['source']
    data_new_list.append(data_new)


In [42]:
class KnowledgeGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def add_solvent(self, name, smiles, density, weight):
        with self.driver.session() as session:
            session.write_transaction(create_solvent_node, name, smiles, density, weight)

    def add_salt(self, name, smiles):
        with self.driver.session() as session:
            session.write_transaction(create_salt_node, name, smiles)

def create_solvent_node(tx, name, smiles, density, weight):
    '''
    name: string, common name of the solvent
    smiles: string, smiles representation of the solvent
    density: float (g/ml)
    weight: float molecular weight of the solvent
    '''
    query = (
        "CREATE (s:Solvent {name: $name, smiles: $smiles, density: $density, weight: $weight})"
    )
    tx.run(query, name=name, smiles=smiles, density=density, weight=weight)

def create_salt_node(tx, name, smiles):
    '''
    name: string, common name of the salt
    smiles: string, smiles representation of the salt
    weight: float molecular weight of the salt
    '''
    query = (
        "CREATE (s:Salt {name: $name, smiles: $smiles})"
    )
    tx.run(query, name=name, smiles=smiles)

def create_electrolyte_node(tx, name, solvents, salts, coulombic_efficiency, reference_id):
    '''
    name: string, common name of the electrolyte
    solvents: a list of tuples [(solvent_name, volume), ...]
    salts: a list of tuples [(salt_name, concentration_mol_per_l), ...]
    coulombic_efficiency: float, Coulombic efficiency of the electrolyte
    '''
    # Create the Electrolyte node
    query = (
        "CREATE (e:Electrolyte {name: $name, coulombic_efficiency: $coulombic_efficiency})"
    )
    tx.run(query, name=name, coulombic_efficiency=coulombic_efficiency)

    # Connect solvents to the Electrolyte node
    for solvent_name, volume in solvents:
        query = (
            "MATCH (e:Electrolyte {name: $name}), (s:Solvent {name: $solvent_name}) "
            "CREATE (e)-[:VOLUME {volume: $volume}]->(s)"
        )
        tx.run(query, name=name, solvent_name=solvent_name, volume=volume)

    # Connect salts to the Electrolyte node
    for salt_name, concentration in salts:
        query = (
            "MATCH (e:Electrolyte {name: $name}), (s:Salt {name: $salt_name}) "
            "CREATE (e)-[:MOL_L {MOL_L: $concentration}]->(s)"
        )
        tx.run(query, name=name, salt_name=salt_name, concentration=concentration)

    # Connect the Electrolyte node to the Reference node
    query = (
        "MATCH (e:Electrolyte {name: $name}), (r:Reference {reference_id: $reference_id}) "
        "CREATE (e)-[:CITED]->(r)"
    )
    tx.run(query, name=name, reference_id=reference_id)

def add_electrolyte(graph, name, solvents, salts, coulombic_efficiency, reference_id):
    with graph.driver.session() as session:
        session.write_transaction(create_electrolyte_node, name, solvents, salts, coulombic_efficiency, reference_id)

def create_reference_node(tx, reference_id, title, content, page_number):
    '''
    reference_id: string, unique identifier of the reference
    title: string, title of the reference
    content: string, content of the reference
    page_number: int, page number of the reference
    '''
    query = (
        "CREATE (r:Reference {reference_id: $reference_id, title: $title, content: $content, page_number: $page_number})"
    )
    # print(reference_id, title, page_number)
    tx.run(query, reference_id=reference_id, title=title, content=content, page_number=page_number)

def add_reference(graph, documents, reference_id):
    title = paper_db.loc[reference_id-1, 'Title']
    with graph.driver.session() as session:
        for doc in documents:
            session.write_transaction(create_reference_node, reference_id, title, doc.page_content, doc.metadata['page'])


def parse_pdf(path, filtering_references = True):
    loader = PyPDFLoader(path)
    documents = loader.load()
    end = len(documents) - 1
    # find the last page that contains the keyword "references" or "bibliography"
    for i in range(len(documents)-1, -1, -1):
        if "references" in documents[i].page_content.lower() or "bibliography" in documents[i].page_content.lower():
            end = i
            break
    if filtering_references:
        documents = documents[:(end+1)]
    # text_splitter = TokenTextSplitter(chunk_size=50, chunk_overlap=5)
    # splitted_documents = text_splitter.split_documents(documents)
    return documents

In [43]:
graph=KnowledgeGraph(KG_URI, KG_USER, KG_PASSWORD)

Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
Failed to write data to connection IPv4Address(('4013e16e.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))


In [44]:
for idx, row in solvent_db.iterrows():
    graph.add_solvent(row['Solvent'].split(' ')[0], row['SMILES'], row['density (g/mL)'], row['MW'])

for idx, row in salt_db.iterrows():
    graph.add_salt(row['Salt'].split(' ')[0], row['SMILES'])

C:\Users\RiceCake\AppData\Local\Temp\ipykernel_2388\978534291.py:69: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_solvent_node, name, smiles, density, weight)
C:\Users\RiceCake\AppData\Local\Temp\ipykernel_2388\978534291.py:73: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_salt_node, name, smiles)


In [45]:
paper_folder = papers_path
papers = os.listdir(paper_folder)
for paper in papers:
    if paper.endswith(".pdf"):
        documents = parse_pdf(f"{paper_folder}/{paper}")
        reference_id = int(paper.split(".")[0])
        add_reference(graph, documents, reference_id)
        

C:\Users\RiceCake\AppData\Local\Temp\ipykernel_2388\978534291.py:96: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_reference_node, reference_id, title, doc.page_content, doc.metadata['page'])


In [46]:
# {'ce': 97.3, 'electrolyte': {'solvent_list': [{'solvent': 'TEP', 'volume_percent': 0.25}, {'solvent': 'BTFE', 'volume_percent': 0.75}], 'salt_list': [{'salt': 'LiFSI', 'mol_L': 1.2}]},
for data in extracted_data_list:
    name = ''
    solvents = []
    salts = []
    for salt in data['electrolyte']['salt_list']:
        if 'salt' not in salt:
            continue
        salt_name = salt['salt'].split(' ')[0]
        if salt['mol_L']:
            name += str(salt['mol_L']) + ' M ' +salt_name + ' '
        else:
            name += salt_name + ' '
        salts.append((salt_name, salt['mol_L']))
    for solvent in data['electrolyte']['solvent_list']:
        s_name = solvent['solvent'].split(' ')[0]
        if 'solvent' not in solvent:
            continue
        if solvent['volume_percent']:
            name += str(solvent['volume_percent']) + ' % ' + s_name + ' '
        else:
            name += s_name + ' '
        solvents.append((s_name, solvent['volume_percent']))
    reference_id = data['source']
    add_electrolyte(graph, name, solvents, salts, data['ce'], int(reference_id))
    
        

C:\Users\RiceCake\AppData\Local\Temp\ipykernel_2388\978534291.py:77: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_electrolyte_node, name, solvents, salts, coulombic_efficiency, reference_id)
